In [329]:
import pandas as pd

In [330]:
train_data = pd.read_csv('salary_train.csv', header=0, sep=',')

train_data.head(3)

,Id,algebra,programming,data science,robotics,economics,job,salary
0,0,87,62,86,61,90,junior developer,140000
1,1,76,84,76,80,79,data scientist,780000
2,2,56,55,99,82,98,developer,210000


In [331]:
predict_data = pd.read_csv('salary_predict.csv', header=0, sep=',')

predict_data.head(3)

,Id,algebra,programming,data science,robotics,economics,job,salary
0,9000,73,59,57,54,61,robotics engineer,0
1,9001,77,80,53,93,80,senior developer,0
2,9002,95,72,88,63,84,developer,0


In [332]:
# проверяю наличие nan
assert train_data.isna().sum().sum() == 0
assert predict_data.isna().sum().sum() == 0

In [333]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            9000 non-null   int64 
 1   algebra       9000 non-null   int64 
 2   programming   9000 non-null   int64 
 3   data science  9000 non-null   int64 
 4   robotics      9000 non-null   int64 
 5   economics     9000 non-null   int64 
 6   job           9000 non-null   object
 7   salary        9000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 562.6+ KB


In [334]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1000 non-null   int64 
 1   algebra       1000 non-null   int64 
 2   programming   1000 non-null   int64 
 3   data science  1000 non-null   int64 
 4   robotics      1000 non-null   int64 
 5   economics     1000 non-null   int64 
 6   job           1000 non-null   object
 7   salary        1000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 62.6+ KB


In [335]:
# заменяю наименования должностей на категорию
train_jobs = train_data.job.unique()
predict_jobs = predict_data.job.unique()

assert len(train_jobs) == len(predict_jobs)

for job in train_jobs:
    assert job in predict_jobs

jobs_dict = dict(zip(train_jobs, range(1, len(train_jobs) + 1)))

train_data['job_category'] = train_data.job.map(jobs_dict)
predict_data['job_category'] = predict_data.job.map(jobs_dict)

In [336]:
train_data.drop(train_data.job.name, axis=1, inplace=True)
train_data.drop(train_data.Id.name, axis=1, inplace=True)

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   algebra       9000 non-null   int64
 1   programming   9000 non-null   int64
 2   data science  9000 non-null   int64
 3   robotics      9000 non-null   int64
 4   economics     9000 non-null   int64
 5   salary        9000 non-null   int64
 6   job_category  9000 non-null   int64
dtypes: int64(7)
memory usage: 492.3 KB


In [337]:
predict_data.drop(predict_data.job.name, axis=1, inplace=True)
ids = predict_data.Id
predict_data.drop(predict_data.Id.name, axis=1, inplace=True)

predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   algebra       1000 non-null   int64
 1   programming   1000 non-null   int64
 2   data science  1000 non-null   int64
 3   robotics      1000 non-null   int64
 4   economics     1000 non-null   int64
 5   salary        1000 non-null   int64
 6   job_category  1000 non-null   int64
dtypes: int64(7)
memory usage: 54.8 KB


In [338]:
# разбиваю данные на X & y

X_train = train_data.drop(train_data.salary.name, axis=1)
y_train = train_data.salary

X_predict = predict_data.drop(predict_data.salary.name, axis=1)

In [339]:
X_train.head(3)

,algebra,programming,data science,robotics,economics,job_category
0,87,62,86,61,90,1
1,76,84,76,80,79,2
2,56,55,99,82,98,3


In [340]:
X_predict.head(3)

,algebra,programming,data science,robotics,economics,job_category
0,73,59,57,54,61,5
1,77,80,53,93,80,6
2,95,72,88,63,84,3


In [341]:
y_train.head(3)

0    140000
1    780000
2    210000
Name: salary, dtype: int64

In [342]:
# особо не парился с выбором модели, RandomForestRegressor подойдет

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [343]:
y_predict = pd.DataFrame(ids)
y_predict['salary'] = model.predict(X_predict)

y_predict.to_csv('salary_submition.csv', sep=',', header=True, index=False)

y_predict

,Id,salary
0,9000,684300.0
1,9001,562600.0
2,9002,332900.0
3,9003,1043900.0
4,9004,149000.0
...,...,...
995,9995,186000.0
996,9996,368000.0
997,9997,333200.0
998,9998,133400.0
